In [27]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [28]:
TRAIN_DIR = "train/"
TEST_DIR = "test/"
IMAGE_SIZE = 50

In [29]:
train_image_file_names = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)][0:3000] 
test_image_file_names = [TEST_DIR+i for i in os.listdir(TEST_DIR)][0:1000]

In [ ]:
def resize_image(image_file_names, height, width):
    images = []
    
    graph = tf.Graph()
    with graph.as_default():
        file_name = tf.placeholder(dtype=tf.string)
        file = tf.read_file(file_name)
        image = tf.image.decode_jpeg(file)
        image = tf.image.resize_image_with_crop_or_pad(image, height, width)
    
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer()  
        for i in range(len(image_file_names)):
            images.append(session.run(image, feed_dict={file_name: image_file_names[i]}))
            if (i+1) % 1000 == 0:
                print('Images processed: ',i+1)
        
        session.close()
    
    return images

In [ ]:
processed_train_images = resize_image(train_image_file_names,500,500)
processed_test_images = resize_image(test_image_file_names, 500, 500)

Images processed:  1000
Images processed:  2000


In [ ]:
for i in range(10):
    plt.imshow(processed_train_images[i])
    plt.show()

In [ ]:
labels = [[1., 0.] if 'dog' in name else [0., 1.] for name in train_image_file_names]

In [ ]:
def resize_and_process_image(filepath):
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_CUBIC)
    return image2

In [ ]:
def resize_and_process_images(image_file_names):
    processed_images =[]
    for image in image_file_names:
        img = resize_and_process_image(image)
        processed_images.append(img)
    return processed_images

In [ ]:
for i in range(20):
    image = resize_and_process_image(train_image_file_names[i])
    print (type(image))
    print (image.shape)
    plt.imshow(image)
    plt.show()

In [ ]:
image_size_flat = IMAGE_SIZE * IMAGE_SIZE
image_shape = (IMAGE_SIZE, IMAGE_SIZE)
num_classes = 2
num_channels = 1
filter_size = 10
num_filters = 5
shape = (filter_size, filter_size, num_channels, num_filters)

In [ ]:

def get_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev = 0.05))

def get_biases(num_filters):
    return tf.Variable(tf.constant(0.05, shape=[num_filters]))


In [ ]:
x = tf.placeholder(tf.float32, shape=[None, image_size_flat], name='x')
x_image = tf.reshape(x, [-1, IMAGE_SIZE, IMAGE_SIZE, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)


In [ ]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = get_weights(shape=shape)
    biases = get_biases(num_filters)
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    layer += biases
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights

In [ ]:
def flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [ ]:
def new_fc_layer(input,         
                 num_inputs,     
                 num_outputs,    
                 use_relu=True): 

    # Create new weights and biases.
    weights = get_weights(shape=[num_inputs, num_outputs])
    biases = get_biases(num_outputs)

    layer = tf.matmul(input, weights) + biases

    
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

In [ ]:
layer_conv1 = new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)
    

In [ ]:
layer_conv1


In [ ]:
layer_flat, num_features = flatten_layer(layer_conv1)

In [ ]:
num_features

In [ ]:
fc_size = 128

In [ ]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [ ]:
layer_fc1

In [ ]:
y_pred = tf.nn.softmax(layer_fc1)

In [ ]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc1,
                                                        labels=y_true)

cost = tf.reduce_mean(cross_entropy)